In [1]:
import cbpro as gdax
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict

from datetime import datetime, timedelta
from math import ceil
from time import sleep

%matplotlib inline
client = gdax.PublicClient()

In [2]:
def historic_data(client, product, num_candles, granularity, max_per=300):
    now = datetime.utcnow().replace(microsecond=0).replace(second=0)
    unit = timedelta(seconds=granularity)
    begin = now - (num_candles * unit)
    delta = unit * max_per
    full = ceil(num_candles / max_per)
    rates = []
    for i in range(0,full):
        frm = begin + (delta * i)
        to  = frm + delta
        chunk = client.get_product_historic_rates(
            product, start=frm.isoformat(), end=to.isoformat(), granularity=granularity)
        if type(chunk) is not list:
            print(chunk)
        else:
            rates.extend(chunk)
            print('From: {}, To: {} - {} points'.format(frm.isoformat(), to.isoformat(), len(chunk)))
        sleep(0.6)
    return rates 

In [3]:
def save_rates(product, num_candles=None, granularity=60):
    rates = historic_data(client=client, product=product, num_candles=num_candles, granularity=granularity)
    print('{} prices'.format(len(rates)))
    # Load into Pandas
    pd_rates = pd.DataFrame(rates)
    # Set column names
    pd_rates.columns = ["Time", "Low", "High", "Open", "Close", "Volume"]
    # Convert Unix time to UTC
    pd_rates["Time"] = pd.to_datetime(pd_rates["Time"],unit='s')
    # Remove duplicates
    before = len(pd_rates)
    pd_rates.drop_duplicates(inplace=True)
    print("Dropped {} duplicate rows".format(before - len(pd_rates)))
    print(pd_rates.head())
    # Save to CSV
    fn = "./historic_data/{}_{}_{}.csv".format(product, granularity, len(pd_rates))
    pd_rates.to_csv(fn, index=False)
    return pd_rates


In [5]:
all_prods = client.get_products()
for prod in all_prods:
    prod_str = prod['id']
    if ('USD' in prod_str and 'USDC' not in prod_str) or ('EUR' in prod_str):
        print("Downloading data for {}".format(prod_str))
        save_rates(prod_str, granularity=60 * 5, num_candles=200000)

From: 2016-12-16T08:52:00, To: 2016-12-17T09:52:00 - 0 points
From: 2016-12-17T09:52:00, To: 2016-12-18T10:52:00 - 0 points
From: 2016-12-18T10:52:00, To: 2016-12-19T11:52:00 - 0 points
From: 2016-12-19T11:52:00, To: 2016-12-20T12:52:00 - 0 points
From: 2016-12-20T12:52:00, To: 2016-12-21T13:52:00 - 0 points
From: 2016-12-21T13:52:00, To: 2016-12-22T14:52:00 - 0 points
From: 2016-12-22T14:52:00, To: 2016-12-23T15:52:00 - 0 points
From: 2016-12-23T15:52:00, To: 2016-12-24T16:52:00 - 0 points
From: 2016-12-24T16:52:00, To: 2016-12-25T17:52:00 - 0 points
From: 2016-12-25T17:52:00, To: 2016-12-26T18:52:00 - 0 points
From: 2016-12-26T18:52:00, To: 2016-12-27T19:52:00 - 0 points
From: 2016-12-27T19:52:00, To: 2016-12-28T20:52:00 - 0 points
From: 2016-12-28T20:52:00, To: 2016-12-29T21:52:00 - 0 points
From: 2016-12-29T21:52:00, To: 2016-12-30T22:52:00 - 0 points
From: 2016-12-30T22:52:00, To: 2016-12-31T23:52:00 - 0 points
From: 2016-12-31T23:52:00, To: 2017-01-02T00:52:00 - 0 points
From: 20